# 1. Web-scrapping data on youtube

In [55]:
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [56]:
# Define url where data is scrapped
url = "https://www.youtube.com/watch?v=zIwLWfaAg-8" #--> epic split ad

In [57]:
# Initialize browser to be controlled by Selenium
serv = Service(executable_path="chromedriver-win64\\chromedriver.exe")
opts = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=serv, options=opts)
driver.get(url)
time.sleep(2)

In [58]:
# Accept terms and conditions
find_terms_accept = driver.find_elements(By.XPATH,"//button[@class='yt-spec-button-shape-next yt-spec-button-shape-next--filled yt-spec-button-shape-next--mono yt-spec-button-shape-next--size-m' and contains(@aria-label,'Accept the use of cookies and other data for the purposes described')]") 
find_terms_accept[0].click() 
time.sleep(2)

# Pause the video --> adapt this code to your local language!!!
find_pause = driver.find_elements(By.XPATH,"//button[@class='ytp-play-button ytp-button'][@data-title-no-tooltip='Pause']") 
find_pause[0].click()
time.sleep(2)

#### Collect meta data 

In [60]:
# Click show more button
btn = driver.find_elements(By.XPATH,"//tp-yt-paper-button[@class='button style-scope ytd-text-inline-expander']")
btn[1].click()

In [61]:
# Publisher
find = driver.find_elements(By.XPATH,"//yt-formatted-string[@class='style-scope ytd-channel-name complex-string']") 
publisher = find[0].text
print(publisher)

TED


In [62]:
# Subscribers
find = driver.find_elements(By.XPATH,"//yt-formatted-string[@class='style-scope ytd-video-owner-renderer']")
subscribers = find[0].text
print(subscribers)

24.6M subscribers


In [63]:
# meta-data: title
find_title = driver.find_elements(By.XPATH,"//yt-formatted-string[@class='style-scope ytd-watch-metadata']")
title = find_title[0].text
print(title)

Elon Musk: The future we're building -- and boring | TED


In [64]:
# meta-data: views, publish date
find_views_pubdate = driver.find_elements(By.XPATH,"//span[@class='style-scope yt-formatted-string bold']")
views = find_views_pubdate[0].text
print(views)

pub_date = find_views_pubdate[2].text
print(pub_date)

26,076,503 views
3 May 2017


In [65]:
# Description
find = driver.find_elements(By.XPATH,"//yt-attributed-string[@class='style-scope ytd-text-inline-expander']")
description_txt = find[0].text
print(description_txt)

Elon Musk discusses his new project digging tunnels under LA, the latest from Tesla and SpaceX and his motivation for building a future on Mars in conversation with TED's Head Curator, Chris Anderson.

Visit http://TED.com to get our entire library of TED Talks, transcripts, translations, personalized talk recommendations and more.

The TED Talks channel features the best talks and performances from the TED Conference, where the world's leading thinkers and doers give the talk of their lives in 18 minutes (or less). Look for talks on Technology, Entertainment and Design -- plus science, business, global issues, the arts and more. You're welcome to link to or embed these videos, forward them to others and share these ideas with people you know. 

Become a TED Member: http://ted.com/membership
Follow TED on Twitter:   / tedtalks  
Like TED on Facebook:   / ted  
Subscribe to our channel:    / ted  

TED's videos may be used for non-commercial purposes under a Creative Commons License, At

In [66]:
# Scroll down to load comments
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
time.sleep(5)  # Wait for the comments to load

# Get number of comments
wait = WebDriverWait(driver, 20)
comments_section = wait.until(
    EC.presence_of_element_located((By.XPATH, '//yt-formatted-string[@class="count-text style-scope ytd-comments-header-renderer"]'))
)
cmts_count = comments_section.text
print(cmts_count)

25,128 Comments


In [67]:
# Get number of Likes
find_likes = driver.find_elements(By.XPATH,"//button[@class='yt-spec-button-shape-next yt-spec-button-shape-next--tonal yt-spec-button-shape-next--mono yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--segmented-start']")
likes = find_likes[0].get_attribute("aria-label")
print(f'{likes} {find_likes[0].text}')

like this video along with 388,996 other people 388K


In [68]:
## Tidy up scraped content
this_row = pd.DataFrame(publisher,index=['Publisher'],columns=['Value'])

new_row = pd.DataFrame(subscribers,index=['Subscriptions'],columns=['Value'])
this_row = pd.concat([this_row,new_row])

new_row = pd.DataFrame(title,index=['Title'],columns=['Value'])
this_row = pd.concat([this_row,new_row])

new_row = pd.DataFrame(views,index=['Views'],columns=['Value'])
this_row = pd.concat([this_row,new_row])

new_row = pd.DataFrame(pub_date,index=['Publish date'],columns=['Value'])
this_row = pd.concat([this_row,new_row])
new_row = pd.DataFrame(description_txt,index=['Description'],columns=['Value'])
this_row = pd.concat([this_row,new_row])
new_row = pd.DataFrame(cmts_count,index=['Comments'],columns=['Value'])
this_row = pd.concat([this_row,new_row])
new_row = pd.DataFrame(likes,index=['Likes'],columns=['Value'])
this_row = pd.concat([this_row,new_row])

meta_df = this_row
print(meta_df)

                                                           Value
Publisher                                                    TED
Subscriptions                                  24.6M subscribers
Title          Elon Musk: The future we're building -- and bo...
Views                                           26,076,503 views
Publish date                                          3 May 2017
Description    Elon Musk discusses his new project digging tu...
Comments                                         25,128 Comments
Likes            like this video along with 388,996 other people


# Setup for comments

In [69]:
# Sort by our choice
# Click "Sort by" to open a dropdown menu
btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[@id='icon-label'][text()='Sort by']"))
)

# Navigate "Top comments", use ActionChains to move to the element
driver.execute_script("arguments[0].scrollIntoView(true);", btn)
actions = ActionChains(driver)
actions.move_to_element(btn).perform()

# Click the element 
driver.execute_script("arguments[0].click();", btn)

# Opening replies

In [70]:
# How many times should it find all the reply buttons and open them
n_runs = 50

for run_nr in range(n_runs):
    # Find all reply buttons
    reply_btn = driver.find_elements(By.XPATH, ".//ytd-button-renderer[contains(@class, 'replies')]")
    print(f"Run {run_nr + 1}")
    print("Opening all reply buttons found")
    print(f"How many reply buttons were found: {len(reply_btn)}")
    
    # Only every second element is an actual button
    reply_btn = reply_btn[::2] 

    # Find first non null element
    for i in range(len(reply_btn)):
        if reply_btn[i].text != "":
            start_index = i
            break
        
    # Open all reply buttons
    for i in range(start_index, len(reply_btn)):
        try:
            reply_btn[i].click()
        except:
            continue
        time.sleep(2)
        html = driver.find_element(By.TAG_NAME, 'html')
        html.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        
        # Click show more button unless there is none
        while True:
            show_more_btn = driver.find_elements(By.XPATH, "//button[@aria-label='Show more replies']")
            if len(show_more_btn) == 0:
                break
            try:
                show_more_btn[0].click()
            except:
                continue
            time.sleep(2)

Run 1
Opening all reply buttons found
How many reply buttons were found: 30
Run 2
Opening all reply buttons found
How many reply buttons were found: 54
Run 3
Opening all reply buttons found
How many reply buttons were found: 82
Run 4
Opening all reply buttons found
How many reply buttons were found: 102
Run 5
Opening all reply buttons found
How many reply buttons were found: 126
Run 6
Opening all reply buttons found
How many reply buttons were found: 144
Run 7
Opening all reply buttons found
How many reply buttons were found: 164
Run 8
Opening all reply buttons found
How many reply buttons were found: 186
Run 9
Opening all reply buttons found
How many reply buttons were found: 206
Run 10
Opening all reply buttons found
How many reply buttons were found: 226
Run 11
Opening all reply buttons found
How many reply buttons were found: 246
Run 12
Opening all reply buttons found
How many reply buttons were found: 246
Run 13
Opening all reply buttons found
How many reply buttons were found: 24

# Scraping username, number of likes and comments

In [71]:
# Get user names
users = []
find = driver.find_elements(By.XPATH,"//div[@id='header-author'][@class='style-scope ytd-comment-view-model']")

for user_name in find:
        users.append(user_name.text)

print(users)

['@raedjah6417\n4 years ago', '@jg63776\n4 years ago', '@thatguynicky1979\n4 years ago', '@ibeezisi3847\n4 years ago', '@leyren2606\n4 years ago (edited)', '@ClaireYunFarronXIII\n4 years ago (edited)', '@Butterpants1987\n4 years ago', '@shoaibhuq8472\n4 years ago (edited)', '@alikhan-uh3sq\n4 years ago', '@LunarXZ_\n4 years ago', '@bambitazilla9464\n4 years ago (edited)', '@theviralmemes7101\n4 years ago', '@alinktoth389\n4 years ago', '@MusicInfected\n4 years ago', '@yougotta2505\n4 years ago', '@DAVIDDESIGN88\n4 years ago', '@DAVIDDESIGN88\n4 years ago', '@aren1998\n4 years ago', '@kcb5989\n4 years ago', '@BharathKumar-uu3fs\n4 years ago', '@polpol1005\n2 years ago', '@junayED.\n2 years ago', '@polpol1005\n2 years ago (edited)', '@samnova4335\n2 years ago', '@martinkabagambe3271\n2 years ago (edited)', '@aadeshnayak.baravo9997\n2 years ago', '@lordofnight9377\n2 years ago', '@roslyntaber9580\n2 years ago', '@teresawilliamson9377\n2 years ago', '@teresawilliamson9377\n2 years ago', '@

In [72]:
# Get users and time stamp
users_clean = []
time_stamp = []
for user in users:
    if user != "":
        try:
            this_ts = user.split('\n')[1]
            this_user = user.split('\n')[0]
        except:
            this_ts = user.split('\n')[0]
            this_user = ""

        users_clean.append(this_user) 
        time_stamp.append(this_ts)

In [73]:
# Get Comments
comments = []

# Get comment caption
find = driver.find_elements(By.XPATH,"//span[@class='yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap'][@dir='auto']")
for comment in find:
    comments.append(comment.text)

In [74]:
# Remove \n strings to avoid wrong display in output
comments_cleaned = []
for cmt in comments[2:-1]:
    clean_cmt = cmt.replace("\n","") #--> replace \n by whitespace
    clean_cmt.strip(" ") #--> remove unncessary whitespaces
    comments_cleaned.append(clean_cmt)

In [75]:
# Get Likes
likes = []

find = driver.find_elements(By.XPATH,"//span[@id='vote-count-middle'][@class='style-scope ytd-comment-engagement-bar']")
for like in find:
    likes.append(like.text)

In [76]:
print("How many elements of each category were scraped:")
print(f"Number of Comments: {len(comments_cleaned)}")
print(f"Number of Timestamps: {len(time_stamp)}")
print(f"Number of Users: {len(users)}")
print(f"Number of Likes: {len(likes)}")

How many elements of each category were scraped:
Number of Comments: 2873
Number of Timestamps: 2873
Number of Users: 2873
Number of Likes: 2873


# Storing the data

In [77]:
# Ensure all lists have the same length
def adjust_length(lst, target_length):
    return lst[:target_length] if len(lst) > target_length else lst + [None] * (target_length - len(lst))

# Use the minimum length of the lists to ensure all are equal length
target_length = min(len(users_clean), len(comments_cleaned), len(time_stamp), len(likes))

users_clean = adjust_length(users_clean, target_length)
comments_cleaned = adjust_length(comments_cleaned, target_length)
time_stamp = adjust_length(time_stamp, target_length)
likes = adjust_length(likes, target_length)

# Store DataFrame
data = {'User': users_clean, 'Comment': comments_cleaned, 'Time': time_stamp, 'Likes': likes}
df = pd.DataFrame(data)
print(df)

                   User                                            Comment  \
0          @raedjah6417                                                      
1              @jg63776     I like how he always says “we” . A real leader   
2     @thatguynicky1979                              Communism intensifies   
3         @ibeezisi3847  We, as in ALL of the geniuses and innovators h...   
4           @leyren2606  Very good observation! I hope to imbibe that m...   
...                 ...                                                ...   
2868     @drijazdurrani  You need to build flood mitigation measures th...   
2869            @1Lex17  "Think of the Future and Don't be SAD"  is a  ...   
2870    @paxtongray3462  I'm more shocked that people don't expect us t...   
2871         @pcm.43441  "I'm not trying to be anyone's savior" Is huma...   
2872       @bceebro1344                       My inspiration is Elone Musk   

                      Time Likes  
0              4 years ago  

In [78]:
# Save info to csv file
df.to_csv('./data/elon_comments.csv', index=False, encoding='utf-8', sep = ";")